In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
from collections import Counter
from fastai.vision.all import *
import albumentations

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
path='/kaggle/input/expertclass2/'

In [1]:
%%time
train_df = pd.read_csv("/kaggle/input/expertclass2/train_data.csv", sep=',')
train_df = pd.concat([train_df['emotion'],train_df['pixels'].str.split(' ', expand=True)], axis=1)
test_df = pd.read_csv("/kaggle/input/expertclass2/test_data.csv", sep=',')
test_df = pd.concat([test_df['pixels'].str.split(' ', expand=True)], axis=1)

In [1]:
submission_df = pd.read_csv("/kaggle/input/expertclass2/example_submission.csv", sep=',')

In [1]:
train_df = train_df.astype(np.uint8)
train_df.head(1)

In [1]:
test_df = test_df.astype(np.uint8)
test_df.head(1)

In [1]:
print(f'Train_df shape : {train_df.shape}')
print(f'Test_df shape  : {test_df.shape}')

In [1]:
def split_df(df):
    '''return a tuple (X, y) 
    
        X : the training inputs which is in (samples, height, width, channel) shape
        y : the label which is flatten
    '''
    if 'emotion' in df.columns:
        y = df['emotion'].values.flatten()
        X = df.drop('emotion', axis=1).values
    else:
        X = df.values
        y = None
        
    X = X.reshape(X.shape[0], 48, 48)
#     X = np.stack((X,) * 3, axis=-1)
    return (X,y)

class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
            
def get_train_aug(): return albumentations.Compose([
            albumentations.Transpose(p=0.2),
            albumentations.VerticalFlip(p=0.2),
            albumentations.ShiftScaleRotate(p=0.2),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(): return albumentations.Compose([
    albumentations.Resize(224,224)
], p=1.)
item_tfms = [Resize(224)]#, AlbumentationsTransform(get_train_aug(), get_valid_aug())]

In [1]:
X_train, y_train = split_df(train_df)
X_test, _   = split_df(test_df)
# X_train = np.stack((X_train,) * 3, axis=-1)
# X_test  = np.stack((X_test,) * 3, axis=-1)

In [1]:
print(f'Train set shape : {X_train.shape, y_train.shape}')
print(f'Test  set shape  : {X_test.shape}')

In [1]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [1]:
index = 3
plt.imshow(X_train[index,], cmap='gray')
plt.title(emotions[y_train[index]]);

In [1]:
# Thanks to nghiaho12 (http://nghiaho.com/?p=2741)

def make_dataloaders_from_numpy_data(image, label):
    def pass_index(idx):
        return idx

    def get_x(i):
        # NOTE: This is a grayscale image that appears to just work with a network expecting RGB.
        # I suspect this is due to tensor broadcasting rules.
        return image[i]

    def get_y(i):
        return label[i]

    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=pass_index,
#         item_tfms=Resize(224),
        batch_tfms=aug_transforms(size=224),
        get_x=get_x,
        get_y=get_y)

    # pass in a list of index
    num_images = image.shape[0]
    dls = dblock.dataloaders(list(range(num_images)), valid_pct=0.2, seed=42)

    return dls

dls = make_dataloaders_from_numpy_data(X_train, y_train)

# sanity check
dls.train.show_batch(max_n=9, cmap='gray')

In [1]:
learn = cnn_learner(dls,
                    models.resnet34,
                    metrics=accuracy,
                    pretrained=False,
                    n_in=1 # 1 channel - grayscale images
                   )

In [1]:
learn.unfreeze()

In [1]:
lr_min, lr_step = learn.lr_find()
print(lr_min, lr_step)

In [1]:
learn.fit_one_cycle(50, lr_max=lr_min*1.1, cbs=[EarlyStoppingCallback(patience=5, monitor='accuracy')])

In [1]:
learn.show_results(cmap='gray')

In [1]:
class_interp = ClassificationInterpretation.from_learner(learn)

In [1]:
class_interp.plot_top_losses(9, figsize=(15,10), cmap='gray')

In [1]:
class_interp.plot_confusion_matrix()

In [1]:
learn.save('fer_model_resnet34')

In [1]:
# predict and submit
# learn.predict()

In [1]:
predictions, *_ = learn.get_preds(dl=dls.test_dl(X_test))
labels = np.argmax(predictions, 1)
Counter(labels.cpu().detach().numpy())

Test Time Augmentation

In [1]:
tta, *_ = learn.tta(dl=dls.test_dl(X_test), use_max=True)
labels_tta = np.argmax(tta, 1)
Counter(labels_tta.cpu().detach().numpy())

Submission

In [1]:
submission_df['Emotion'] = labels_tta
submission_df.to_csv("submission.csv", index=False)
submission_df